In [ ]:
%%R
# ---------------------------
# Partie 1 : Classification synthétique
# ---------------------------

# Charger les données d'entraînement
data <- read.table("synth_train.txt", header = TRUE)
data_test <- read.table("synth_test.txt", header = TRUE)

# Vérifier les valeurs manquantes
if (sum(is.na(data)) > 0) {
  print(paste("Attention:", sum(is.na(data)), "valeurs manquantes détectées dans les données d'entraînement"))
}

if (sum(is.na(data_test)) > 0) {
  print(paste("Attention:", sum(is.na(data_test)), "valeurs manquantes détectées dans les données de test"))
}

# Convertir y en facteur
data$y <- as.factor(data$y)
data_test$y <- as.factor(data_test$y)

# Vérification des dimensions
print(paste("Dimensions des données d'apprentissage :", dim(data)[1], "x", dim(data)[2]))
print(paste("Dimensions des données de test :", dim(data_test)[1], "x", dim(data_test)[2]))

# Construire l'arbre de classification standard
library(rpart)
library(rpart.plot)

X <- data[, !(names(data) %in% "y")]
y <- data$y

tree_standard <- rpart(y ~ ., data = data, method = "class")

# Visualiser l'arbre
png("arbre_standard_synth.png", width = 2000, height = 1000, res = 300)
prp(tree_standard, extra = 1, faclen = 0, fallen.leaves = TRUE, shadow.col = "gray",
    main = "Arbre de classification Standard (Synthétique)")
dev.off()

# Calcul des erreurs
pred_train_standard <- predict(tree_standard, data, type = "class")
error_train_standard <- 1 - mean(pred_train_standard == data$y)

pred_test_standard <- predict(tree_standard, data_test, type = "class")
error_test_standard <- 1 - mean(pred_test_standard == data_test$y)

print(paste("Erreur d'apprentissage (Standard) :", round(error_train_standard, 4)))
print(paste("Erreur de test (Standard) :", round(error_test_standard, 4)))

# Matrice de confusion
conf_matrix <- table(data_test$y, pred_test_standard)
print("Matrice de confusion - Arbre standard:")
print(conf_matrix)

# Arbre avec profondeur maximale
tree_max_depth <- rpart(y ~ ., data = data, method = "class",
                       control = rpart.control(maxdepth = 5))

# Visualiser l'arbre
png("arbre_max_depth_synth.png", width = 2000, height = 1000, res = 300)
prp(tree_max_depth, extra = 1, faclen = 0, fallen.leaves = TRUE, shadow.col = "gray",
    main = "Arbre de classification avec Profondeur Maximale (Synthétique)")
dev.off()

# Calcul des erreurs
pred_train_max_depth <- predict(tree_max_depth, data, type = "class")
error_train_max_depth <- 1 - mean(pred_train_max_depth == data$y)

pred_test_max_depth <- predict(tree_max_depth, data_test, type = "class")
error_test_max_depth <- 1 - mean(pred_test_max_depth == data_test$y)

print(paste("Erreur d'apprentissage (Profondeur Maximale) :", round(error_train_max_depth, 4)))
print(paste("Erreur de test (Profondeur Maximale) :", round(error_test_max_depth, 4)))

# Matrice de confusion
conf_matrix_md <- table(data_test$y, pred_test_max_depth)
print("Matrice de confusion - Arbre avec profondeur maximale:")
print(conf_matrix_md)

# ---------------------------
# Partie 1 : Classification synthétique (inchangée)
# ---------------------------

# -------------------------------
# 2. Reconnaissance de chiffres manuscrits - OPTIMISÉ
# -------------------------------

# Charger les données ZIP avec data.table (plus rapide)
library(data.table)
zip_train <- fread("zip_train.txt", header = FALSE)
zip_test <- fread("zip_test.txt", header = FALSE)

# Nommer les colonnes
colnames(zip_train) <- c("y", paste0("x", 1:256))
colnames(zip_test) <- c("y", paste0("x", 1:256))

# Convertir y en facteur (plus rapide en utilisant les niveaux connus)
zip_train[, y := factor(y, levels = 0:9)]
zip_test[, y := factor(y, levels = 0:9)]

# Fonction optimisée pour afficher des exemples d'images
display_images <- function(data) {
  unique_classes <- 0:9
  par(mfrow = c(2, 5), mar = c(0.5, 0.5, 2, 0.5))

  for (cls in unique_classes) {
    cls_data <- data[y == cls]
    if (nrow(cls_data) > 0) {
      sample_row <- as.matrix(cls_data[sample(.N, 1), -1])
      image(matrix(sample_row, 16),
           col = gray(255:0/255),
           axes = FALSE,
           main = paste("Classe", cls))
    }
  }
}

# Visualisation plus rapide en réduisant la résolution
png("digit_samples.png", width = 800, height = 500)
display_images(zip_train)
dev.off()

# Paramètres optimisés pour l'arbre
tree_params <- rpart.control(
  maxdepth = 10,         # Réduit la profondeur maximale
  minsplit = 20,         # Augmente le seuil de division
  minbucket = 10,        # Augmente la taille minimale des feuilles
  cp = 0.01,             # Seuil de complexité plus élevé
  maxcompete = 0,        # Désactive les variables competitives
  maxsurrogate = 0,      # Désactive les variables surrogate
  usesurrogate = 0,      # Désactive l'utilisation de surrogate
  xval = 5               # Réduit le nombre de folds pour validation
)

# Entraînement de l'arbre avec paramètres optimisés
system.time({
  tree_zip <- rpart(y ~ .,
                   data = zip_train,
                   method = "class",
                   control = tree_params)
})

# Arbre simplifié pour visualisation
tree_zip_simple <- rpart(y ~ .,
                        data = zip_train,
                        method = "class",
                        control = rpart.control(maxdepth = 3,
                                              minsplit = 100,
                                              minbucket = 50))

# Visualisation plus rapide
png("arbre_zip_simple.png", width = 1200, height = 800)
prp(tree_zip_simple,
    extra = 1,
    faclen = 0,
    fallen.leaves = TRUE,
    shadow.col = "gray",
    main = "Arbre simplifié pour chiffres manuscrits",
    branch.lty = 3,
    nn = FALSE)
dev.off()

# Prédictions optimisées
pred_zip_test <- predict(tree_zip, zip_test, type = "class")

# Calcul des erreurs avec data.table (plus rapide)
error_zip_train <- 1 - mean(predict(tree_zip, zip_train, type = "class") == zip_train$y)
error_zip_test <- 1 - mean(pred_zip_test == zip_test$y)

# Matrice de confusion avec caret (plus lisible)
library(caret)
conf_matrix <- confusionMatrix(pred_zip_test, zip_test$y)
print(conf_matrix$table)

# Erreur par classe optimisée
error_by_class <- zip_test[, .(Error = 1 - mean(pred_zip_test == y)), by = y]
print("Erreur par chiffre:")
print(error_by_class[order(y)])

# Paires les plus confondues (méthode optimisée)
conf_pairs <- as.data.table(as.data.frame(conf_matrix$table))
top_confusions <- conf_pairs[Prediction != Reference][order(-Freq), .(Reference, Prediction, Freq)][1:5]
print("Top 5 des confusions:")
print(top_confusions)

[1] "Dimensions des données d'apprentissage : 100 x 3"
[1] "Dimensions des données de test : 200 x 3"
[1] "Erreur d'apprentissage (Standard) : 0.08"
[1] "Erreur de test (Standard) : 0.19"
[1] "Matrice de confusion - Arbre standard:"
   pred_test_standard
      1   2
  1  25  37
  2   1 137
[1] "Erreur d'apprentissage (Profondeur Maximale) : 0.08"
[1] "Erreur de test (Profondeur Maximale) : 0.19"
[1] "Matrice de confusion - Arbre avec profondeur maximale:"
   pred_test_max_depth
      1   2
  1  25  37
  2   1 137


In [ ]:
%%R
# Code pour installer les bibliothèques nécessaires pour l'analyse par arbres de décision

# Vérifier si une bibliothèque est déjà installée et l'installer si nécessaire
install_if_missing <- function(package) {
  if (!require(package, character.only = TRUE, quietly = TRUE)) {
    message(paste("Installation du package:", package))
    install.packages(package, dependencies = TRUE)
    if (!require(package, character.only = TRUE, quietly = TRUE)) {
      warning(paste("Échec de l'installation du package:", package))
    } else {
      message(paste("Package", package, "installé et chargé avec succès"))
    }
  } else {
    message(paste("Package", package, "déjà installé et chargé"))
  }
}

# Liste des packages nécessaires
packages <- c(
  "readr",      # Pour la lecture de fichiers
  "dplyr",      # Pour la manipulation de données
  "ggplot2",    # Pour les visualisations
  "rpart",      # Pour les arbres de décision
  "rpart.plot", # Pour visualiser les arbres
  "caret"       # Pour l'évaluation des modèles
)

# Installer et charger chaque package
for (pkg in packages) {
  install_if_missing(pkg)
}

# Vérifier que tous les packages sont disponibles
all_installed <- sapply(packages, require, character.only = TRUE)
if (all(all_installed)) {
  message("Tous les packages nécessaires sont installés et prêts à être utilisés.")
} else {
  missing_pkgs <- packages[!all_installed]
  warning(paste("Les packages suivants n'ont pas pu être installés:",
                paste(missing_pkgs, collapse = ", ")))
}

# Afficher les versions des packages installés
if (all(all_installed)) {
  installed_versions <- sapply(packages, function(p) as.character(packageVersion(p)))
  package_info <- data.frame(
    Package = packages,
    Version = installed_versions,
    stringsAsFactors = FALSE
  )
  print(package_info)
}

# Notifier que l'installation est terminée
message("Configuration de l'environnement de travail terminée.")

              Package Version
readr           readr   2.1.5
dplyr           dplyr   1.1.4
ggplot2       ggplot2   3.5.2
rpart           rpart  4.1.24
rpart.plot rpart.plot   3.1.2
caret           caret   7.0.1


Package readr déjà installé et chargé

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Package dplyr déjà installé et chargé
Package ggplot2 déjà installé et chargé
Package rpart déjà installé et chargé
Installation du package: rpart.plot
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘plotrix’, ‘Formula’, ‘plotmo’, ‘earth’

trying URL 'https://cran.rstudio.com/src/contrib/plotrix_3.8-4.tar.gz'
Content type 'application/x-gzip' length 318991 bytes (311 KB)
downloaded 311 KB

trying URL 'https://cran.rstudio.com/src/contrib/Formula_1.2-5.tar.gz'
Content type 'application/x-gzip' length 128259 bytes (125 KB)
downloaded 125 KB

trying URL 'https://cran.rstudio.com/src/contrib/plotmo_3.6.4.tar.gz'
Content type 'application/x-gzip' length 1263769 bytes (1.2 MB)
downloaded 1.2 M

In [ ]:

%load_ext rpy2.ipython